In [1]:
try:
    import psycopg2
    HAS_POSTGRES = False #para forzar uso de dummy data
except ImportError:
    HAS_POSTGRES = False
    print("psycopg2 no instalado. Se usará modo offline (CSV).")

import pandas as pd
import geopandas as gpd
import unicodedata
import warnings
from pathlib import Path
from shapely import wkb

# Ignorar advertencias
warnings.filterwarnings('ignore')

# ==========================================
# 0. FUNCIONES
# ==========================================

def safe_read_parquet(path):
    """Lee archivos parquet con manejo robusto."""
    try:
        return pd.read_parquet(path, engine='pyarrow')
    except Exception as e:
        print(f"Error PyArrow: {e}. Intentando fastparquet...")
        return pd.read_parquet(path, engine='fastparquet')

def normalize_text(text):
    if isinstance(text, str):
        text = text.strip().upper()
        return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    return text

# ==========================================
# 1. CARGA DE PUNTOS BLUE (SQL o CSV)
# ==========================================

print("--- Paso 1: Descargando Puntos Blue ---")

DB_HOST = "dwh.datarq.blue.internal"
DB_NAME = "dwh"
DB_USER = "molivares" 
DB_PASS = "MA2012"
CSV_PATH = 'data/dummy_data.csv'

PUDOS_Bx = None

# Intentar cargar desde SQL
if HAS_POSTGRES:
    try:
        print("Intentando conectar a PostgreSQL...")
        con = psycopg2.connect(database=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST, port="5432")
        cur = con.cursor()
        sql = "SELECT cmns_nmb, geol_latitud, geol_longitud, estado,esto_seq_cdg FROM reports.pickup_maestro_v02 WHERE estado = 'active';"
        cur.execute(sql)
        rows = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        con.close()
        
        PUDOS_Bx = pd.DataFrame(rows, columns=columns)
        print(f"Datos cargados desde SQL: {len(PUDOS_Bx)} registros.")
        
    except Exception as e:
        print(f"Error SQL: {e}")
        PUDOS_Bx = None
else:
    print("PostgreSQL no disponible (psycopg2 no instalado).")

# Si falló SQL o no hay driver, intentar CSV
if PUDOS_Bx is None:
    print(f"Intentando cargar desde CSV: {CSV_PATH} ...")
    try:
        PUDOS_Bx = pd.read_csv(CSV_PATH)
        print(f"Datos cargados desde CSV: {len(PUDOS_Bx)} registros.")
    except Exception as e:
        print(f"Error cargando CSV: {e}")
        PUDOS_Bx = pd.DataFrame() # DataFrame vacío

# Procesar si hay datos
if not PUDOS_Bx.empty:
    try:
        # Limpieza coordenadas
        if 'geol_latitud' in PUDOS_Bx.columns and 'geol_longitud' in PUDOS_Bx.columns:
            PUDOS_Bx['geol_latitud'] = PUDOS_Bx['geol_latitud'].astype(str).str.replace(',', '.').astype(float)
            PUDOS_Bx['geol_longitud'] = PUDOS_Bx['geol_longitud'].astype(str).str.replace(',', '.').astype(float)
            
            # Crear GeoDataFrame PUDO
            gdf_PUDO = gpd.GeoDataFrame(PUDOS_Bx, geometry=gpd.points_from_xy(PUDOS_Bx.geol_longitud, PUDOS_Bx.geol_latitud), crs="EPSG:4326")
            
            if 'cmns_nmb' in PUDOS_Bx.columns:
                gdf_PUDO['COMUNA_NORM'] = gdf_PUDO['cmns_nmb'].apply(normalize_text)
            else:
                print("Advertencia: Columna 'cmns_nmb' no encontrada. Buscando alternativas...")
                if 'COMUNA' in PUDOS_Bx.columns:
                     gdf_PUDO['COMUNA_NORM'] = PUDOS_Bx['COMUNA'].apply(normalize_text)
                else:
                     gdf_PUDO['COMUNA_NORM'] = ''
            
            print(f"GeoDataFrame creado: {len(gdf_PUDO)} puntos.")
        else:
             print("Error: El CSV no tiene columnas 'geol_latitud' y 'geol_longitud'.")
             gdf_PUDO = gpd.GeoDataFrame(columns=['cmns_nmb', 'geometry'], crs="EPSG:4326")

    except Exception as e:
        print(f"Error procesando datos: {e}")
        gdf_PUDO = gpd.GeoDataFrame(columns=['cmns_nmb', 'geometry'], crs="EPSG:4326")
else:
    print("No se pudieron cargar datos (ni SQL ni CSV). Se usará GeoDataFrame vacío.")
    gdf_PUDO = gpd.GeoDataFrame(columns=['cmns_nmb', 'geometry'], crs="EPSG:4326")

# ==========================================
# 2. CARGA DE CENSO (URBANO + RURAL)
# ==========================================

print("--- Paso 2: Cargando Censo 2024 (Urbano y Rural) ---")

path_manzanas = 'Cartografia_censo2024_Pais/Cartografia_censo2024_Pais_Manzanas.parquet'
use_unified_parquet = True
unified_path = 'Cartografia_censo2024_Pais\Censo2024_Unificado_Nacional.parquet'
rural_extra = gpd.GeoDataFrame(columns=['geometry'], crs='EPSG:4326')

df_manzanas = safe_read_parquet(path_manzanas)

# Geometría base
col_geometria = 'geometry'
if 'SHAPE' in df_manzanas.columns:
    col_geometria = 'SHAPE'

if df_manzanas[col_geometria].dtype == 'object' or isinstance(df_manzanas[col_geometria].iloc[0], bytes):
    print("Convirtiendo WKB...")
    df_manzanas[col_geometria] = df_manzanas[col_geometria].apply(lambda x: wkb.loads(x) if isinstance(x, bytes) else x)

manzanas_base = gpd.GeoDataFrame(df_manzanas, geometry=col_geometria)
if manzanas_base.crs is None:
    manzanas_base.set_crs(epsg=4326, inplace=True)

# Población base
if 'n_per' in manzanas_base.columns:
    manzanas_base.rename(columns={'n_per': 'TOTAL_PERS'}, inplace=True)
elif 'TOTAL_PERS' not in manzanas_base.columns:
    manzanas_base['TOTAL_PERS'] = 0

manzanas_base['TIPO_FUENTE'] = 'URBANO'
frames_censo = [manzanas_base]

if use_unified_parquet and Path(unified_path).exists():
    print(f"--- Cargando parquet unificado: {unified_path} ---")
    df_unif = safe_read_parquet(unified_path)

    geom_col = None
    if 'geometry' in df_unif.columns:
        geom_col = 'geometry'
    elif 'SHAPE' in df_unif.columns:
        geom_col = 'SHAPE'
    else:
        for col in df_unif.columns:
            if 'geom' in col.lower() or 'shape' in col.lower():
                geom_col = col
                break

    if geom_col is None:
        print("Advertencia: parquet unificado sin columna de geometría. Se mantiene manzanas base.")
    else:
        if df_unif[geom_col].dtype == 'object' or isinstance(df_unif[geom_col].iloc[0], bytes):
            print("Convirtiendo WKB del parquet unificado...")
            df_unif[geom_col] = df_unif[geom_col].apply(lambda x: wkb.loads(x) if isinstance(x, bytes) else x)

        gdf_unif = gpd.GeoDataFrame(df_unif, geometry=geom_col)
        if gdf_unif.crs is None:
            gdf_unif.set_crs(epsg=4326, inplace=True)

        gdf_unif['TIPO_REGISTRO'] = gdf_unif['TIPO_REGISTRO'].fillna('')
        if 'TOTAL_PERS' not in gdf_unif.columns and 'n_per' in gdf_unif.columns:
            gdf_unif.rename(columns={'n_per': 'TOTAL_PERS'}, inplace=True)
        if 'MANZENT' not in gdf_unif.columns and 'UID' in gdf_unif.columns:
            gdf_unif['MANZENT'] = gdf_unif['UID']

        gdf_urban = gdf_unif[gdf_unif['TIPO_REGISTRO'] == 'MANZANA_URBANA'].copy()
        rural_extra = gdf_unif[gdf_unif['TIPO_REGISTRO'] != 'MANZANA_URBANA'].copy()

        if not gdf_urban.empty:
            gdf_urban['TIPO_FUENTE'] = 'URBANO'
            frames_censo = [gdf_urban]
        else:
            print("Advertencia: el parquet unificado no trae MANZANA_URBANA. Se conserva manzanas base.")

        if not rural_extra.empty:
            rural_extra['TIPO_FUENTE'] = 'RURAL'
            if 'MANZENT' not in rural_extra.columns and 'UID' in rural_extra.columns:
                rural_extra['MANZENT'] = rural_extra['UID']
            if 'TOTAL_PERS' not in rural_extra.columns and 'n_per' in rural_extra.columns:
                rural_extra.rename(columns={'n_per': 'TOTAL_PERS'}, inplace=True)
            frames_censo.append(rural_extra)
            print(f"Rural agregado desde parquet unificado: {len(rural_extra):,} filas")

manzanas = pd.concat(frames_censo, ignore_index=True)
print(f"Censo cargado: urbano+rural {len(manzanas):,} registros")

# ==========================================
# 3. PREPARACIÓN DE DATOS (NIVEL MANZANA/LOCALIDAD)
# ==========================================

print("--- Paso 3: Preparando Datos (Nivel Manzana/Localidad) ---")

manzanas_full = manzanas.copy()

# 1. Asegurar ID Único de Manzana (MANZENT)
if 'MANZENT' in manzanas_full.columns:
    # Handle both numeric and string MANZENT values
    if pd.api.types.is_numeric_dtype(manzanas_full['MANZENT']):
        manzanas_full['MANZENT'] = manzanas_full['MANZENT'].fillna(0).astype('int64').astype(str)
    else:
        manzanas_full['MANZENT'] = manzanas_full['MANZENT'].fillna('UNKNOWN').astype(str)
else:
    print("ADVERTENCIA: No se encontró MANZENT. Creando ID sintético.")
    manzanas_full['MANZENT'] = manzanas_full.index.astype(str)

# 2. Rellenar Nulos en Jerarquía
cols_fill = {
    'ID_ENTIDAD': 0,
    'ENTIDAD': 'DESCONOCIDA',
    'CATEGORIA': 'DESCONOCIDA',
    'LOCALIDAD': 'DESCONOCIDA',
    'AREA_C': 'DESCONOCIDA',
    'COMUNA': 'DESCONOCIDA',
    'TIPO_FUENTE': 'URBANO'
}

for col, val in cols_fill.items():
    if col not in manzanas_full.columns:
        manzanas_full[col] = val
    else:
        if col == 'ID_ENTIDAD':
             manzanas_full[col] = manzanas_full[col].fillna(0).astype('int64')
        else:
             manzanas_full[col] = manzanas_full[col].fillna(val)

# Ajuste específico para ID_ENTIDAD=0 -> Rural
mask_rural = manzanas_full['ID_ENTIDAD'] == 0
manzanas_full.loc[mask_rural, 'ENTIDAD'] = 'DISPERSO / RURAL'
manzanas_full.loc[mask_rural, 'CATEGORIA'] = 'DISPERSO'

# Normalizar Comuna
manzanas_full['COMUNA_NORM'] = manzanas_full['COMUNA'].apply(normalize_text)

# Proyección UTM
print("Proyectando a UTM 19S...")
manzanas_utm = manzanas_full.to_crs(epsg=32719)
gdf_pudo_utm = gdf_PUDO.to_crs(epsg=32719)

# Para geometrías puntuales (típicas en rural), agregar un pequeño buffer para permitir cálculo de área/cobertura
is_point_geom = manzanas_utm.geometry.geom_type.isin(['Point', 'MultiPoint'])
if is_point_geom.any():
    manzanas_utm.loc[is_point_geom, 'geometry'] = manzanas_utm.loc[is_point_geom].geometry.buffer(1)

# ==========================================
# 4. CÁLCULO DE COBERTURA POR MANZANA/LOCALIDAD
# ==========================================

print("--- Paso 4: Calculando Cobertura por Manzana/Localidad ---")

resultados_manzanas = []
grupos_comuna = manzanas_utm.groupby('COMUNA_NORM')
total = len(grupos_comuna)
count = 0

for comuna_norm, df_mz_comuna in grupos_comuna:
    count += 1
    if count % 20 == 0:
        print(f"Procesando Comuna {count}/{total}: {comuna_norm}")

    # Filtrar PUDOs
    pudos_comuna = gdf_pudo_utm[gdf_pudo_utm['COMUNA_NORM'] == comuna_norm]
    
    # Preparar datos base de la comuna
    df_mz_comuna = df_mz_comuna.copy()
    df_mz_comuna['area_total_m2'] = df_mz_comuna.geometry.area
    df_mz_comuna['area_total_m2'] = df_mz_comuna['area_total_m2'].replace(0, 1.0)
    
    # Columnas a guardar
    cols_output = [
        'MANZENT', 'COMUNA', 'ENTIDAD', 'CATEGORIA', 'LOCALIDAD', 'AREA_C', 'TIPO_FUENTE',
        'TOTAL_PERS', 'area_total_m2', 'ID_ENTIDAD', 'ID_LOCALIDAD'
    ]
    cols_validas = [c for c in cols_output if c in df_mz_comuna.columns]
    
    # Indexar por MANZENT para mapeo rápido
    base_data = df_mz_comuna[cols_validas].set_index('MANZENT')
    
    # Inicializar métricas
    base_data['area_cubierta_m2'] = 0.0
    base_data['pudos_en_comuna'] = len(pudos_comuna)
    base_data['pudos_en_manzana'] = 0

    if not pudos_comuna.empty:
        try:
            # --- A. CÁLCULO DE COBERTURA (ÁREA) ---
            buffer_union = pudos_comuna.geometry.buffer(800).unary_union
            gdf_buffer = gpd.GeoDataFrame(geometry=[buffer_union], crs=df_mz_comuna.crs)
            
            interseccion = gpd.overlay(df_mz_comuna, gdf_buffer, how='intersection')
            
            if not interseccion.empty:
                interseccion['area_pedazo'] = interseccion.geometry.area
                areas_cubiertas = interseccion.groupby('MANZENT')['area_pedazo'].sum()
                base_data.loc[areas_cubiertas.index, 'area_cubierta_m2'] = areas_cubiertas
            
            # --- B. CONTEO DE PUDOS CERCANOS (Por Manzana/Localidad) ---
            pudos_buffers = pudos_comuna.copy()
            pudos_buffers['geometry'] = pudos_buffers.geometry.buffer(800)
            
            geom_col_name = df_mz_comuna.geometry.name
            mz_for_sjoin = df_mz_comuna[['MANZENT', geom_col_name]].copy()
            mz_for_sjoin = gpd.GeoDataFrame(mz_for_sjoin, geometry=geom_col_name, crs=df_mz_comuna.crs)
            
            joined_pudos = gpd.sjoin(mz_for_sjoin, pudos_buffers[['geometry']], how='inner', predicate='intersects')
            
            counts_pudos = joined_pudos.groupby('MANZENT').size()
            base_data.loc[counts_pudos.index, 'pudos_en_manzana'] = counts_pudos

        except Exception as e:
            print(f"Error en comuna {comuna_norm}: {e}")

    # Calcular Porcentajes y Población
    base_data['pct_cobertura'] = (base_data['area_cubierta_m2'] / base_data['area_total_m2']).fillna(0.0)
    base_data['pct_cobertura'] = base_data['pct_cobertura'].clip(upper=1.0)
    
    base_data['pob_cubierta'] = base_data['TOTAL_PERS'] * base_data['pct_cobertura']
    
    resultados_manzanas.append(base_data.reset_index())

# ==========================================
# 5. CONSOLIDACIÓN Y GUARDADO
# ==========================================

print("--- Paso 5: Guardando Reporte Granular ---")

if resultados_manzanas:
    df_final = pd.concat(resultados_manzanas, ignore_index=True)
    
    df_final['pob_cubierta'] = df_final['pob_cubierta'].round(2)
    df_final['pct_cobertura'] = df_final['pct_cobertura'].round(4)
    df_final['area_cubierta_m2'] = df_final['area_cubierta_m2'].round(1)
    df_final['area_total_m2'] = df_final['area_total_m2'].round(1)
    
    df_final['pudos_en_manzana'] = df_final['pudos_en_manzana'].astype(int)
    df_final['pudos_en_comuna'] = df_final['pudos_en_comuna'].astype(int)
    
    df_final = df_final.sort_values(by=['COMUNA', 'ENTIDAD', 'MANZENT'])
    
    filename = 'Cobertura_Censo24_Por_Manzana.xlsx'
    
    print(f"Generando Excel con {len(df_final)} filas...")
    #df_final.to_excel(filename, index=False)
    print(f"Archivo generado: {filename}")
else:
    print("No se generaron resultados.")

<>:121: SyntaxWarning: invalid escape sequence '\C'
<>:121: SyntaxWarning: invalid escape sequence '\C'
C:\Users\martin.olivares\AppData\Local\Temp\ipykernel_28492\2314124282.py:121: SyntaxWarning: invalid escape sequence '\C'
  unified_path = 'Cartografia_censo2024_Pais\Censo2024_Unificado_Nacional.parquet'


--- Paso 1: Descargando Puntos Blue ---
PostgreSQL no disponible (psycopg2 no instalado).
Intentando cargar desde CSV: data/dummy_data.csv ...
Datos cargados desde CSV: 3424 registros.
GeoDataFrame creado: 3424 puntos.
--- Paso 2: Cargando Censo 2024 (Urbano y Rural) ---
Convirtiendo WKB...
--- Cargando parquet unificado: Cartografia_censo2024_Pais\Censo2024_Unificado_Nacional.parquet ---
Convirtiendo WKB del parquet unificado...
Rural agregado desde parquet unificado: 28,415 filas
Censo cargado: urbano+rural 244,756 registros
--- Paso 3: Preparando Datos (Nivel Manzana/Localidad) ---
Proyectando a UTM 19S...
--- Paso 4: Calculando Cobertura por Manzana/Localidad ---
Procesando Comuna 20/345: CALBUCO
Procesando Comuna 40/345: CHEPICA
Procesando Comuna 60/345: COLTAUCO
Procesando Comuna 80/345: DALCAHUE
Procesando Comuna 100/345: GORBEA
Procesando Comuna 120/345: LA FLORIDA
Procesando Comuna 140/345: LINARES
Procesando Comuna 160/345: MACHALI
Procesando Comuna 180/345: NATALES
Procesand

In [2]:
df_final['TOTAL_PERS'].sum()

np.float64(18226208.0)

In [3]:
len(manzanas)

244756

In [4]:
# ==========================================
# 6. GENERACIÓN DE REPORTE AGRUPADO (ALDEA/LOCALIDAD)
# ==========================================

print("--- Paso 6: Generando Reporte Agrupado (Aldea/Localidad) ---")

if 'df_final' in locals() and not df_final.empty:
    print("Agrupando datos por COMUNA y LOCALIDAD (sin AREA_C)...")
    
    cols_group = ['COMUNA', 'LOCALIDAD']
    
    df_agrupado = df_final.groupby(cols_group).agg({
        'TOTAL_PERS': 'sum',
        'pob_cubierta': 'sum',
        'area_total_m2': 'sum',
        'area_cubierta_m2': 'sum',
        'pudos_en_comuna': 'first'
    }).reset_index()
    
    df_agrupado['PORCENTAJE_COBERTURA'] = (df_agrupado['pob_cubierta'] / df_agrupado['TOTAL_PERS']).fillna(0)

    # Identificación única de PUDOs
    gdf_pudo_counts = gdf_pudo_utm.copy()
    id_col = 'esto_seq_cdg' if 'esto_seq_cdg' in gdf_pudo_counts.columns else ('PUDO_ID' if 'PUDO_ID' in gdf_pudo_counts.columns else None)
    
    if id_col is None:
        gdf_pudo_counts['PUDO_ID_WORK'] = (-1000 * pd.Series(gdf_pudo_counts.index, index=gdf_pudo_counts.index)).astype(int)
    else:
        gdf_pudo_counts['PUDO_ID_WORK'] = gdf_pudo_counts[id_col].fillna((-1000 * pd.Series(gdf_pudo_counts.index, index=gdf_pudo_counts.index)).astype(int))
    
    gdf_pudo_counts = gdf_pudo_counts.drop_duplicates(subset=['PUDO_ID_WORK'])
    gdf_pudo_counts['COMUNA_NORM'] = gdf_pudo_counts['COMUNA_NORM'].fillna('').astype(str)
    
    # --- CÁLCULO DE PUDOS POR LOCALIDAD ---
    print("Calculando PUDOs por Localidad (solo puntos dentro de la comuna)...")
    
    pudos_por_localidad = {}
    extra_rows = []
    
    # Diccionario maestro de totales reales
    pudos_por_comuna_real = {}

    for comuna in df_agrupado['COMUNA'].unique():
        comuna_norm = normalize_text(comuna)
        
        # 1. Obtener PUDOs base por nombre normalizado
        pudos_comuna_base = gdf_pudo_counts[gdf_pudo_counts['COMUNA_NORM'] == comuna_norm][['PUDO_ID_WORK', 'geometry']].copy()
        
        # Guardar total teórico inicial
        pudos_por_comuna_real[comuna_norm] = len(pudos_comuna_base)

        mz_comuna_full = manzanas_utm[manzanas_utm['COMUNA_NORM'] == comuna_norm].copy()
        
        if mz_comuna_full.empty or pudos_comuna_base.empty:
            continue
            
        geom_col_mz = mz_comuna_full.geometry.name or 'geometry'

        # 2. Rellenar LOCALIDAD si falta
        if 'LOCALIDAD' not in mz_comuna_full.columns:
             mz_comuna_full['LOCALIDAD'] = 'DESCONOCIDA'
        mz_comuna_full['LOCALIDAD'] = mz_comuna_full['LOCALIDAD'].fillna('DESCONOCIDA').astype(str)

        # 3. Filtrar columnas necesarias
        cols_select = list(set(['LOCALIDAD', geom_col_mz]))
        mz_comuna = gpd.GeoDataFrame(mz_comuna_full[cols_select].copy(), geometry=geom_col_mz, crs=mz_comuna_full.crs)

        try:
            # 4. Asignación Espacial (Nearest)
            # Primero intentamos con radio 1km
            pudos_con_localidad = gpd.sjoin_nearest(
                pudos_comuna_base[['PUDO_ID_WORK', 'geometry']],
                mz_comuna,
                how='left',
                max_distance=1000
            )

            # Fallback para los que quedaron NaN (más lejos de 1km) -> Asignar al más cercano sin límite
            if pudos_con_localidad['LOCALIDAD'].isna().any():
                missing_mask = pudos_con_localidad['LOCALIDAD'].isna()
                missing_ids = pudos_con_localidad.loc[missing_mask, 'PUDO_ID_WORK'].unique()
                
                missing_pudos = pudos_comuna_base[pudos_comuna_base['PUDO_ID_WORK'].isin(missing_ids)]
                
                fallback_matches = gpd.sjoin_nearest(
                    missing_pudos[['PUDO_ID_WORK', 'geometry']],
                    mz_comuna,
                    how='left'
                )
                
                # Actualizar los valores NaN con los del fallback
                # Mapear ID -> Localidad del fallback
                fallback_map = fallback_matches.set_index('PUDO_ID_WORK')['LOCALIDAD'].to_dict()
                pudos_con_localidad['LOCALIDAD'] = pudos_con_localidad.apply(
                    lambda row: fallback_map.get(row['PUDO_ID_WORK'], row['LOCALIDAD']) if pd.isna(row['LOCALIDAD']) else row['LOCALIDAD'],
                    axis=1
                )

            # Relleno final por seguridad
            pudos_con_localidad['LOCALIDAD'] = pudos_con_localidad['LOCALIDAD'].fillna('SIN_LOCALIDAD')

            # 5. Conteo Único
            pudos_unique_loc = pudos_con_localidad[['PUDO_ID_WORK', 'LOCALIDAD']].drop_duplicates(subset=['PUDO_ID_WORK'])
            conteo = pudos_unique_loc.groupby('LOCALIDAD')['PUDO_ID_WORK'].nunique()
            
            # Guardar en diccionario global
            for locality, count in conteo.items():
                pudos_por_localidad[(comuna, str(locality))] = int(count)

        except Exception as e:
            print(f"  Error procesando PUDOs en {comuna}: {e}")

    # --- INYECCIÓN DE LOCALIDADES FALTANTES ---
    # Verificar si hay claves en pudos_por_localidad que NO están en df_agrupado
    # Esto ocurre si un PUDO cae en una localidad que no tenía manzanas en el reporte final (raro) o por discrepancia de nombres
    
    existing_keys = set(zip(df_agrupado['COMUNA'], df_agrupado['LOCALIDAD'].astype(str)))
    new_rows = []
    
    for (c_key, l_key), count_val in pudos_por_localidad.items():
        if (c_key, l_key) not in existing_keys:
            # Crear fila nueva
            print(f"  > Agregando localidad faltante al reporte: {c_key} - {l_key} ({count_val} PUDOs)")
            new_rows.append({
                'COMUNA': c_key,
                'LOCALIDAD': l_key,
                'TOTAL_PERS': 0,
                'pob_cubierta': 0,
                'area_total_m2': 0,
                'area_cubierta_m2': 0,
                'pudos_en_comuna': pudos_por_comuna_real.get(normalize_text(c_key), 0),
                'CANTIDAD_PUDOS_LOCALIDAD': count_val
            })

    if new_rows:
        df_new = pd.DataFrame(new_rows)
        df_agrupado = pd.concat([df_agrupado, df_new], ignore_index=True)

    # --- ASIGNACIÓN FINAL DE CONTEOS ---
    # Iterar nuevamente para asegurar que todos tengan valor (los existentes y los nuevos)
    for idx, row in df_agrupado.iterrows():
        c_key = row['COMUNA']
        l_key = str(row['LOCALIDAD'])
        
        # Asignar conteo
        cnt = pudos_por_localidad.get((c_key, l_key), 0)
        df_agrupado.at[idx, 'CANTIDAD_PUDOS_LOCALIDAD'] = cnt
        
        # Asignar total comunal real
        cnorm = normalize_text(c_key)
        df_agrupado.at[idx, 'pudos_en_comuna'] = pudos_por_comuna_real.get(cnorm, 0)

    # Formateo final
    cols_check = ['CANTIDAD_PUDOS_LOCALIDAD', 'pudos_en_comuna', 'TOTAL_PERS', 'pob_cubierta']
    for col in cols_check:
        df_agrupado[col] = df_agrupado[col].fillna(0).astype(int)

    df_agrupado['PORCENTAJE_COBERTURA'] = (df_agrupado['pob_cubierta'] / df_agrupado['TOTAL_PERS'].replace(0, 1)).fillna(0).round(4)
    
    # Diagnóstico de cuadratura
    mismatch_rows = []
    for comuna in df_agrupado['COMUNA'].unique():
        sub = df_agrupado[df_agrupado['COMUNA'] == comuna]
        total_real = sub['pudos_en_comuna'].iloc[0] # Debería ser igual en todas las filas de la comuna
        suma_loc = sub['CANTIDAD_PUDOS_LOCALIDAD'].sum()
        
        if total_real != suma_loc:
            mismatch_rows.append({
                'COMUNA': comuna,
                'REAL': total_real,
                'SUMA_LOC': suma_loc,
                'DIF': total_real - suma_loc
            })
            
    if mismatch_rows:
        mismatch_df = pd.DataFrame(mismatch_rows).sort_values(by='DIF', ascending=False)
        print("\n⚠️  ALERTA DE DESCUADRE (Top 10):")
        print(mismatch_df.head(10))
    else:
        print("\n✅  Todas las comunas cuadran perfectamente (PUDOs Comuna vs Suma Localidades).")

    # Columnas internas (usando los nombres reales del DataFrame)
    cols_final_internal = [
        'COMUNA', 'LOCALIDAD',
        'TOTAL_PERS', 'pob_cubierta', 'PORCENTAJE_COBERTURA',
        'CANTIDAD_PUDOS_LOCALIDAD', 'pudos_en_comuna'
    ]
    
    df_resumen = df_agrupado[cols_final_internal].sort_values(by=['COMUNA', 'LOCALIDAD'])
    
    # Renombrar para exportación (User-Friendly names)
    df_resumen = df_resumen.rename(columns={
        'TOTAL_PERS': 'POBLACION_TOTAL',
        'pob_cubierta': 'POBLACION_CUBIERTA'
    })
    
    filename_resumen = 'Cobertura_Censo24_Resumen_Localidad.xlsx'
    try:
        df_resumen.to_excel(filename_resumen, index=False)
    except PermissionError:
        alt_name = 'Cobertura_Censo24_Resumen_Localidad_alt.xlsx'
        print(f"Aviso: '{filename_resumen}' está bloqueado. Guardando en '{alt_name}'")
        df_resumen.to_excel(alt_name, index=False)
        filename_resumen = alt_name
    print(f"Archivo de resumen generado: {filename_resumen}")

else:
    print("Error: No se encontró el DataFrame 'df_final'. Ejecuta el paso anterior primero.")


--- Paso 6: Generando Reporte Agrupado (Aldea/Localidad) ---
Agrupando datos por COMUNA y LOCALIDAD (sin AREA_C)...
Calculando PUDOs por Localidad (solo puntos dentro de la comuna)...

✅  Todas las comunas cuadran perfectamente (PUDOs Comuna vs Suma Localidades).
Archivo de resumen generado: Cobertura_Censo24_Resumen_Localidad.xlsx


In [5]:
# ==========================================
# 6.1 GENERACIÓN DE REPORTE POR PUDO (+ PARTNERS)
# ==========================================

print("--- Paso 6.1: Generando reporte por PUDO (incluyendo Partners) ---")

if gdf_PUDO is None or gdf_PUDO.empty:
    print("No se encontraron PUDOs para procesar.")
elif 'manzanas_utm' not in locals():
    print("Error: faltan datos de manzanas (ejecute pasos anteriores).")
else:
    # Preparación de insumos - VERSION MEJORADA CON VALIDACIÓN (sin AREA_C)
    cols_needed = ['MANZENT', 'TOTAL_PERS', 'LOCALIDAD', 'COMUNA', 'COMUNA_NORM', 'geometry']
    cols_available = [c for c in cols_needed if c in manzanas_utm.columns]
    geom_col = manzanas_utm.geometry.name
    if geom_col not in cols_available:
        cols_available.append(geom_col)
    
    try:
        manzanas_work = gpd.GeoDataFrame(
            manzanas_utm[cols_available].copy(),
            geometry=geom_col,
            crs=manzanas_utm.crs
        )
    except Exception as e:
        print(f"Error seleccionando columnas de manzanas_utm: {e}")
        print(f"Columnas disponibles: {list(manzanas_utm.columns)}")
        print("Abortando Paso 6.1")
        manzanas_work = None
    
    if manzanas_work is not None and not manzanas_work.empty:
        geom_col_work = manzanas_work.geometry.name
        manzanas_work['area_m2'] = manzanas_work.geometry.area
        manzanas_work['area_m2'] = manzanas_work['area_m2'].replace(0, 1.0)
        manzanas_area_map = manzanas_work.set_index('MANZENT')['area_m2']
        
        # --- Cargar catálogo de agencias (lat/lon reales y tipo_punto) ---
        try:
            agencias_raw = pd.read_csv("data/partners_blue_capilaridad_qos.csv")
            agencias_raw['geol_latitud'] = agencias_raw['geol_latitud'].astype(str).str.replace(',', '.').astype(float)
            agencias_raw['geol_longitud'] = agencias_raw['geol_longitud'].astype(str).str.replace(',', '.').astype(float)
            agencias_raw['ofcn_dsc'] = agencias_raw.get('ofcn_dsc', '').astype(str).str.strip()
            agencias_raw['q_dao'] = pd.to_numeric(agencias_raw.get('q_dao', 0), errors='coerce').fillna(0)
            
            gdf_ag_wgs = gpd.GeoDataFrame(
                agencias_raw,
                geometry=gpd.points_from_xy(agencias_raw.geol_longitud, agencias_raw.geol_latitud),
                crs="EPSG:4326"
            )
            gdf_ag_wgs['COMUNA_NORM'] = gdf_ag_wgs['cmns_nmb'].apply(normalize_text)
            gdf_ag_wgs['tipo_punto_std'] = gdf_ag_wgs['tipo_punto'].astype(str).str.strip().str.lower()
            gdf_ag_wgs['lat_round'] = gdf_ag_wgs.geometry.y.round(6)
            gdf_ag_wgs['lon_round'] = gdf_ag_wgs.geometry.x.round(6)
            gdf_ag_wgs = gdf_ag_wgs.sort_values('agencyid').drop_duplicates(subset=['lat_round', 'lon_round'], keep='first')
            gdf_ag_utm = gdf_ag_wgs.to_crs(epsg=32719)
        except Exception as e:
            print(f"Error cargando catálogo de agencias: {e}")
            gdf_ag_wgs = gpd.GeoDataFrame(columns=['agencyid', 'tipo_punto_std', 'geometry'], crs="EPSG:4326")
            gdf_ag_utm = gpd.GeoDataFrame(columns=['agencyid', 'tipo_punto_std', 'geometry'], crs="EPSG:32719")
        
        # --- NUEVO: Cargar Partners desde CSV ---
        try:
            print("Cargando Partners desde CSV...")
            partners_raw = pd.read_csv("data/partners_mapa.csv")
            partners_raw['lat'] = pd.to_numeric(partners_raw.get('lat', 0), errors='coerce').fillna(0)
            partners_raw['lon'] = pd.to_numeric(partners_raw.get('lon', 0), errors='coerce').fillna(0)
            partners_raw = partners_raw[(partners_raw['lat'] != 0) & (partners_raw['lon'] != 0)]
            
            if not partners_raw.empty:
                gdf_partners_wgs = gpd.GeoDataFrame(
                    partners_raw,
                    geometry=gpd.points_from_xy(partners_raw.lon, partners_raw.lat),
                    crs="EPSG:4326"
                )
                gdf_partners_wgs['lat_round'] = gdf_partners_wgs.geometry.y.round(6)
                gdf_partners_wgs['lon_round'] = gdf_partners_wgs.geometry.x.round(6)
                gdf_partners_wgs = gdf_partners_wgs.drop_duplicates(subset=['lat_round', 'lon_round'], keep='first')
                gdf_partners_utm = gdf_partners_wgs.to_crs(epsg=32719)
                gdf_partners_utm['OFCN_DSC'] = gdf_partners_utm.get('Partner/Tipo', '').astype(str).str.strip()
                gdf_partners_utm['Q_DAO'] = 0
                gdf_partners_utm['TIPO_PUNTO'] = 'partner'
                print(f"   Partners cargados: {len(gdf_partners_utm)}")
            else:
                gdf_partners_utm = gpd.GeoDataFrame(columns=['geometry', 'OFCN_DSC', 'Q_DAO', 'TIPO_PUNTO'], crs="EPSG:32719")
                print("   No se encontraron partners con coordenadas válidas")
        except Exception as e:
            print(f"Error cargando partners: {e}")
            gdf_partners_utm = gpd.GeoDataFrame(columns=['geometry', 'OFCN_DSC', 'Q_DAO', 'TIPO_PUNTO'], crs="EPSG:32719")
        
        # --- Procesar PUDOs base ---
        gdf_pudo_work = gdf_pudo_utm.copy()
        if 'esto_seq_cdg' in gdf_pudo_work.columns:
            gdf_pudo_work['PUDO_ID'] = gdf_pudo_work['esto_seq_cdg']
        elif 'PUDO_ID' in gdf_pudo_work.columns:
            gdf_pudo_work['PUDO_ID'] = gdf_pudo_work['PUDO_ID']
        else:
            gdf_pudo_work['PUDO_ID'] = (-1000 * pd.Series(gdf_pudo_work.index, index=gdf_pudo_work.index)).astype(int)
        gdf_pudo_work['PUDO_ID'] = gdf_pudo_work['PUDO_ID'].fillna((-1000 * pd.Series(gdf_pudo_work.index, index=gdf_pudo_work.index)).astype(int))
        
        gdf_pudo_work['COMUNA'] = gdf_pudo_work.get('cmns_nmb', gdf_pudo_work.get('COMUNA', ''))
        gdf_pudo_work['COMUNA'] = gdf_pudo_work['COMUNA'].fillna('').astype(str)
        if 'COMUNA_NORM' not in gdf_pudo_work.columns:
            gdf_pudo_work['COMUNA_NORM'] = gdf_pudo_work['COMUNA'].apply(normalize_text)
        else:
            gdf_pudo_work['COMUNA_NORM'] = gdf_pudo_work['COMUNA_NORM'].apply(lambda x: normalize_text(str(x)) if pd.notna(x) else '')
        gdf_pudo_work['COMUNA_NORM'] = gdf_pudo_work['COMUNA_NORM'].astype(str)
        
        gdf_pudo_wgs = gdf_PUDO.copy()
        gdf_pudo_wgs['lat'] = gdf_pudo_wgs.geometry.y
        gdf_pudo_wgs['lon'] = gdf_pudo_wgs.geometry.x
        gdf_pudo_wgs['lat_round'] = gdf_pudo_wgs['lat'].round(6)
        gdf_pudo_wgs['lon_round'] = gdf_pudo_wgs['lon'].round(6)
        pudo_coords = gdf_pudo_wgs[['lat', 'lon', 'lat_round', 'lon_round']].copy()
        pudo_coords['PUDO_ID'] = gdf_pudo_work['PUDO_ID'].values
        gdf_pudo_work = gdf_pudo_work.merge(
            pudo_coords[['PUDO_ID', 'lat', 'lon', 'lat_round', 'lon_round']],
            on='PUDO_ID',
            how='left'
        )
        
        if not gdf_ag_wgs.empty:
            gdf_pudo_work = gdf_pudo_work.merge(
                gdf_ag_wgs[['agencyid', 'tipo_punto_std', 'q_dao', 'ofcn_dsc', 'lat_round', 'lon_round']],
                on=['lat_round', 'lon_round'],
                how='left',
                suffixes=('', '_AG')
            )
        
        if not gdf_ag_utm.empty:
            faltantes_mask = gdf_pudo_work['agencyid'].isna()
            if faltantes_mask.any():
                pudo_missing = gdf_pudo_work.loc[faltantes_mask, ['PUDO_ID', 'geometry']].copy()
                nearest_match = gpd.sjoin_nearest(
                    gpd.GeoDataFrame(pudo_missing, geometry='geometry', crs=gdf_pudo_work.crs),
                    gdf_ag_utm[['agencyid', 'tipo_punto_std', 'q_dao', 'ofcn_dsc', 'geometry']],
                    how='left',
                    max_distance=10
                )
                for _, r in nearest_match.iterrows():
                    idx_pid = r['PUDO_ID']
                    gdf_pudo_work.loc[gdf_pudo_work['PUDO_ID'] == idx_pid, 'agencyid'] = r.get('agencyid')
                    gdf_pudo_work.loc[gdf_pudo_work['PUDO_ID'] == idx_pid, 'tipo_punto_std'] = r.get('tipo_punto_std')
                    gdf_pudo_work.loc[gdf_pudo_work['PUDO_ID'] == idx_pid, 'q_dao'] = r.get('q_dao')
                    gdf_pudo_work.loc[gdf_pudo_work['PUDO_ID'] == idx_pid, 'ofcn_dsc'] = r.get('ofcn_dsc')
        
        gdf_pudo_work['AGENCY_ID'] = gdf_pudo_work['agencyid']
        gdf_pudo_work['TIPO_PUNTO_PUDO'] = gdf_pudo_work['tipo_punto_std']
        gdf_pudo_work['OFCN_DSC'] = gdf_pudo_work.get('ofcn_dsc', '').fillna('').astype(str)
        gdf_pudo_work['Q_DAO'] = gdf_pudo_work.get('q_dao', 0).fillna(0)
        gdf_pudo_work['IS_PARTNER'] = False
        
        if not gdf_partners_utm.empty:
            gdf_partners_utm['PUDO_ID'] = range(-1, -len(gdf_partners_utm)-1, -1)
            gdf_partners_utm['AGENCY_ID'] = None
            gdf_partners_utm['TIPO_PUNTO_PUDO'] = 'partner'
            gdf_partners_utm['lat'] = gdf_partners_wgs.geometry.y.values
            gdf_partners_utm['lon'] = gdf_partners_wgs.geometry.x.values
            gdf_partners_utm['lat_round'] = gdf_partners_wgs['lat_round'].values
            gdf_partners_utm['lon_round'] = gdf_partners_wgs['lon_round'].values
            gdf_partners_utm['COMUNA'] = ''
            gdf_partners_utm['COMUNA_NORM'] = ''
            gdf_partners_utm['IS_PARTNER'] = True
            
            cols_mz_select = ['LOCALIDAD', 'COMUNA', 'COMUNA_NORM', 'geometry']
            cols_mz_avail = [c for c in cols_mz_select if c in manzanas_work.columns]
            if geom_col_work not in cols_mz_avail and geom_col_work in manzanas_work.columns:
                cols_mz_avail.append(geom_col_work)
            
            if cols_mz_avail:
                partners_for_join = gpd.GeoDataFrame(
                    gdf_partners_utm[['PUDO_ID', 'geometry']].copy(),
                    geometry='geometry',
                    crs=gdf_partners_utm.crs
                )
                manzanas_for_join = gpd.GeoDataFrame(
                    manzanas_work[cols_mz_avail].copy(),
                    geometry=geom_col_work,
                    crs=manzanas_work.crs
                )
                nearest_partner = gpd.sjoin_nearest(
                    partners_for_join,
                    manzanas_for_join,
                    how='left',
                    max_distance=800
                )
                cols_merge_p = [c for c in ['PUDO_ID', 'LOCALIDAD', 'COMUNA', 'COMUNA_NORM'] if c in nearest_partner.columns]
                if cols_merge_p:
                    gdf_partners_utm = gdf_partners_utm.merge(
                        nearest_partner[cols_merge_p],
                        on='PUDO_ID',
                        how='left',
                        suffixes=('', '_MZ')
                    )
                    # Deduplicar por ID para evitar expansión
                    gdf_partners_utm = gdf_partners_utm.drop_duplicates(subset=['PUDO_ID'])

                    if 'COMUNA_MZ' in gdf_partners_utm.columns:
                        gdf_partners_utm['COMUNA'] = gdf_partners_utm['COMUNA_MZ'].fillna('').astype(str)
                        gdf_partners_utm.drop(columns=['COMUNA_MZ'], inplace=True)
                    if 'COMUNA_NORM_MZ' in gdf_partners_utm.columns:
                        gdf_partners_utm['COMUNA_NORM'] = gdf_partners_utm['COMUNA_NORM_MZ'].fillna('').astype(str)
                        gdf_partners_utm.drop(columns=['COMUNA_NORM_MZ'], inplace=True)
        
        cols_common_ext = ['PUDO_ID', 'AGENCY_ID', 'OFCN_DSC', 'Q_DAO', 'TIPO_PUNTO_PUDO', 'lat', 'lon', 
                          'lat_round', 'lon_round', 'COMUNA', 'COMUNA_NORM', 'IS_PARTNER', 
                          'LOCALIDAD_PUDO', 'geometry']
        
        if 'LOCALIDAD' in gdf_pudo_work.columns:
            gdf_pudo_work['LOCALIDAD_PUDO'] = gdf_pudo_work['LOCALIDAD'].fillna('DESCONOCIDA').astype(str)
        elif 'LOCALIDAD_PUDO' in gdf_pudo_work.columns:
            gdf_pudo_work['LOCALIDAD_PUDO'] = gdf_pudo_work['LOCALIDAD_PUDO'].fillna('DESCONOCIDA').astype(str)
        else:
            gdf_pudo_work['LOCALIDAD_PUDO'] = 'DESCONOCIDA'
        
        cols_pudo_final = [c for c in cols_common_ext if c in gdf_pudo_work.columns]
        cols_partner_final = [c for c in cols_common_ext if c in gdf_partners_utm.columns]
        
        gdf_all_points = pd.concat([
            gdf_pudo_work[cols_pudo_final],
            gdf_partners_utm[cols_partner_final]
        ], ignore_index=True)
        gdf_all_points = gpd.GeoDataFrame(gdf_all_points, geometry='geometry', crs=gdf_pudo_work.crs)
        print(f"Total puntos a analizar (PUDOs + Partners): {len(gdf_all_points)}")
        
        cols_mz_select = ['LOCALIDAD', 'COMUNA', 'COMUNA_NORM', 'geometry']
        cols_mz_avail = [c for c in cols_mz_select if c in manzanas_work.columns]
        if geom_col_work not in cols_mz_avail and geom_col_work in manzanas_work.columns:
            cols_mz_avail.append(geom_col_work)
        
        if cols_mz_avail and not gdf_all_points.empty:
            missing_localidad = gdf_all_points['LOCALIDAD_PUDO'] == 'DESCONOCIDA'
            if missing_localidad.any():
                points_for_join = gpd.GeoDataFrame(
                    gdf_all_points.loc[missing_localidad, ['PUDO_ID', 'geometry']].copy(),
                    geometry='geometry',
                    crs=gdf_all_points.crs
                )
                manzanas_for_join = gpd.GeoDataFrame(
                    manzanas_work[cols_mz_avail].copy(),
                    geometry=geom_col_work,
                    crs=manzanas_work.crs
                )
                nearest_localidad = gpd.sjoin_nearest(
                    points_for_join,
                    manzanas_for_join,
                    how='left',
                    max_distance=800
                )
                cols_merge = [c for c in ['PUDO_ID', 'LOCALIDAD', 'COMUNA'] if c in nearest_localidad.columns]
                if cols_merge:
                    for idx, row in nearest_localidad.iterrows():
                        pid = row['PUDO_ID']
                        mask = gdf_all_points['PUDO_ID'] == pid
                        if 'LOCALIDAD' in row and pd.notna(row['LOCALIDAD']):
                            gdf_all_points.loc[mask, 'LOCALIDAD_PUDO'] = row['LOCALIDAD']
                        if 'COMUNA' in row and pd.notna(row['COMUNA']):
                            gdf_all_points.loc[mask, 'COMUNA'] = row['COMUNA']
                            gdf_all_points.loc[mask, 'COMUNA_NORM'] = normalize_text(row['COMUNA'])
        
        gdf_all_points['COMUNA'] = gdf_all_points['COMUNA'].fillna('').astype(str)
        gdf_all_points['LOCALIDAD_PUDO'] = gdf_all_points['LOCALIDAD_PUDO'].fillna('DESCONOCIDA').astype(str)
        
        registros_pudo = []
        
        comunas_all = gdf_all_points['COMUNA_NORM'].fillna('').astype(str).unique()
        total_comunas = len(comunas_all)
        anchor_agency_map = gdf_all_points.set_index('PUDO_ID')['AGENCY_ID'].to_dict()
        ofcn_map = gdf_all_points.set_index('PUDO_ID')['OFCN_DSC'].to_dict()
        q_dao_map = gdf_all_points.set_index('PUDO_ID')['Q_DAO'].to_dict()
        
        for idx_c, comuna_norm in enumerate(comunas_all, start=1):
            if idx_c % 20 == 0:
                print(f"   Procesando comuna {idx_c}/{total_comunas}: {comuna_norm}")
            points_comuna = gdf_all_points[gdf_all_points['COMUNA_NORM'] == comuna_norm].copy()
            mz_comuna = manzanas_work[manzanas_work['COMUNA_NORM'] == comuna_norm][['MANZENT', 'TOTAL_PERS', geom_col_work]].copy()
            mz_comuna = gpd.GeoDataFrame(mz_comuna, geometry=geom_col_work, crs=manzanas_work.crs)
            if points_comuna.empty or mz_comuna.empty:
                continue
            
            buffers = points_comuna[['PUDO_ID', 'COMUNA', 'COMUNA_NORM', 'LOCALIDAD_PUDO', 'AGENCY_ID']].copy()
            buffers['center_geom'] = points_comuna.geometry # Guardar centro original
            buffers['geometry'] = points_comuna.geometry.buffer(800)
            buffers = gpd.GeoDataFrame(buffers, geometry='geometry', crs=points_comuna.crs)
            
            inter_all = gpd.overlay(
                mz_comuna[['MANZENT', 'TOTAL_PERS', geom_col_work]],
                buffers[['PUDO_ID', 'geometry']],
                how='intersection'
            )
            if not inter_all.empty:
                inter_all = gpd.GeoDataFrame(inter_all, geometry=inter_all.geometry, crs=mz_comuna.crs)
                inter_all['area_inter'] = inter_all.geometry.area
                inter_all['area_mz'] = inter_all['MANZENT'].map(manzanas_area_map).replace(0, 1.0)
                inter_all['pob_piece'] = inter_all['TOTAL_PERS'] * (inter_all['area_inter'] / inter_all['area_mz'])
                poblacion_total = inter_all.groupby('PUDO_ID')['pob_piece'].sum()
                
                geom_inter = inter_all.geometry.name
                cover_counts = gpd.sjoin(
                    inter_all[['PUDO_ID', 'MANZENT', 'pob_piece', geom_inter]],
                    buffers[['PUDO_ID', 'geometry']],
                    how='left',
                    predicate='intersects'
                )
                cover_counts['covers'] = cover_counts.groupby(cover_counts.index)['PUDO_ID_right'].transform('nunique')
                exclusivos = cover_counts[cover_counts['covers'] == 1]
                poblacion_unica = exclusivos.groupby('PUDO_ID_left')['pob_piece'].sum() if not exclusivos.empty else pd.Series(dtype=float)
            else:
                poblacion_total = pd.Series(dtype=float)
                poblacion_unica = pd.Series(dtype=float)
            
            sindex = buffers.sindex
            vecinos_overlap = {}
            cannibal_potential = {}
            
            for idx_buf, buf_row in buffers.iterrows():
                vecinos_idx = sindex.query(buf_row.geometry, predicate='intersects')
                vecinos_overlap[buf_row.PUDO_ID] = max(len(vecinos_idx) - 1, 0)
                
                total_robbed = 0.0
                vecinos_geoms = buffers.iloc[vecinos_idx]
                
                for _, vecino in vecinos_geoms.iterrows():
                    if vecino.PUDO_ID == buf_row.PUDO_ID:
                        continue
                    intersection = buf_row.geometry.intersection(vecino.geometry)
                    if not intersection.is_empty:
                        area_inter = intersection.area
                        area_vecino = vecino.geometry.area
                        if area_vecino > 0:
                            pct_overlap = area_inter / area_vecino
                            robbed = q_dao_map.get(vecino.PUDO_ID, 0) * pct_overlap
                            total_robbed += robbed
                cannibal_potential[buf_row.PUDO_ID] = total_robbed
            
            q_tipo = {}
            q_vecinos_reales = {}
            
            # --- CONTEO DE PUDOS AZULES/COPEC ---
            if not gdf_ag_utm.empty:
                join_ag = gpd.sjoin(
                    buffers[['PUDO_ID', 'AGENCY_ID', 'geometry']],
                    gdf_ag_utm[['agencyid', 'tipo_punto_std', 'geometry']],
                    how='inner',
                    predicate='intersects'
                )
                if not join_ag.empty:
                    join_ag = join_ag.rename(columns={'PUDO_ID_left': 'PUDO_ID'})
                    join_ag['anchor_agency'] = join_ag['PUDO_ID'].map(anchor_agency_map)
                    join_ag = join_ag[join_ag['agencyid'] != join_ag['anchor_agency']]
                    grouped = join_ag.groupby('PUDO_ID')['tipo_punto_std'].value_counts()
                    for (pid, tipo), val in grouped.items():
                        if pid not in q_tipo:
                            q_tipo[pid] = {'blue': 0, 'copec': 0, 'partner': 0}
                        if tipo == 'blue':
                            q_tipo[pid]['blue'] += int(val)
                        elif tipo == 'copec':
                            q_tipo[pid]['copec'] += int(val)
                        else:
                            q_tipo[pid]['partner'] += int(val)
            
            # --- CONTEO DE PARTNERS (CON AJUSTE -1) ---
            if not gdf_partners_utm.empty:
                ag_partners = gdf_partners_utm.copy()
                ag_partners['agencyid'] = ag_partners['PUDO_ID']
                join_partners = gpd.sjoin(
                    ag_partners[['agencyid', 'lat_round', 'lon_round', 'geometry']],
                    buffers[['PUDO_ID', 'center_geom', 'geometry']],
                    how='inner',
                    predicate='intersects'
                )
                if not join_partners.empty:
                    join_partners = join_partners.rename(columns={'PUDO_ID_right': 'PUDO_ID'})
                    
                    # 1. Filtro por ID (Evitar auto-referencia)
                    join_partners['agencyid'] = join_partners['agencyid'].astype(str)
                    join_partners['PUDO_ID_str'] = join_partners['PUDO_ID'].astype(str)
                    join_partners = join_partners[join_partners['agencyid'] != join_partners['PUDO_ID_str']]
                    
                    # 2. Filtro de "Hijos" de Agencia
                    join_partners['anchor_agency'] = join_partners['PUDO_ID'].map(anchor_agency_map)
                    join_partners = join_partners[join_partners['agencyid'] != join_partners['anchor_agency'].astype(str)]
                    
                    # 3. Triple check: Distancia > 1m para evitar duplicados "fantasma"
                    join_partners['dist_check'] = join_partners.geometry.distance(join_partners['center_geom'])
                    join_partners = join_partners[join_partners['dist_check'] > 1.0]

                    # Deduplicación final
                    join_partners = join_partners.drop_duplicates(subset=['PUDO_ID', 'lat_round', 'lon_round'])
                    partner_counts = join_partners.groupby('PUDO_ID').size()
                    
                    for pid, val in partner_counts.items():
                        if pid not in q_tipo:
                            q_tipo[pid] = {'blue': 0, 'copec': 0, 'partner': 0}
                        
                        # AJUSTE SOLICITADO: Restar 1 al conteo de partners si es mayor a 0
                        final_val = int(val)
                        if final_val > 0:
                            final_val -= 1
                        
                        q_tipo[pid]['partner'] = final_val
            
            for pid, counts in q_tipo.items():
                total_neighbors = counts.get('blue', 0) + counts.get('copec', 0) + counts.get('partner', 0)
                q_vecinos_reales[pid] = total_neighbors
            
            for _, buf_row in buffers.iterrows():
                pid = buf_row.PUDO_ID
                point_info = gdf_all_points[gdf_all_points['PUDO_ID'] == pid].iloc[0]
                lat = point_info.get('lat')
                lon = point_info.get('lon')
                counts_tipo = q_tipo.get(pid, {'blue': 0, 'copec': 0, 'partner': 0})
                ofcn_val = ofcn_map.get(pid, '')
                q_dao_val = q_dao_map.get(pid, 0)
                potencial_robo = cannibal_potential.get(pid, 0.0)
                
                registros_pudo.append({
                    'PUDO_ID': pid,
                    'AGENCY_ID': anchor_agency_map.get(pid),
                    'OFCN_DSC': ofcn_val,
                    'Q_DAO_PROPIO': q_dao_val,
                    'POTENCIAL_ROBO_Q': round(potencial_robo, 2),
                    'COMUNA': buf_row.COMUNA,
                    'LOCALIDAD': buf_row.LOCALIDAD_PUDO,
                    'LAT': lat,
                    'LON': lon,
                    'POBLACION_CUBIERTA_PUDO': int(round(poblacion_total.get(pid, 0))),
                    'POBLACION_UNICA_PUDO': int(round(poblacion_unica.get(pid, 0))),
                    'PUDOS_OVERLAP_BUFFERS': vecinos_overlap.get(pid, 0),
                    'q_vecinos_reales': q_vecinos_reales.get(pid, 0),
                    'q_blue': counts_tipo.get('blue', 0),
                    'q_copec': counts_tipo.get('copec', 0),
                    'q_partner': counts_tipo.get('partner', 0),
                    'TIPO_PUNTO': point_info.get('TIPO_PUNTO_PUDO', ''),
                    'IS_PARTNER': point_info.get('IS_PARTNER', False)
                })
        
        if registros_pudo:
            df_reporte_pudo = pd.DataFrame(registros_pudo)
            filename_pudo = 'Cobertura_Por_PUDO_y_Partners.xlsx'
            try:
                df_reporte_pudo.to_excel(filename_pudo, index=False)
            except PermissionError:
                alt_name = 'Cobertura_Por_PUDO_y_Partners_alt.xlsx'
                print(f"Aviso: '{filename_pudo}' está bloqueado. Guardando en '{alt_name}'")
                df_reporte_pudo.to_excel(alt_name, index=False)
                filename_pudo = alt_name
            print(f"\n✅ Archivo generado: {filename_pudo}")
            print(f"   Total registros: {len(df_reporte_pudo):,}")
            print(f"   - PUDOs: {(~df_reporte_pudo['IS_PARTNER']).sum():,}")
            print(f"   - Partners: {df_reporte_pudo['IS_PARTNER'].sum():,}")
            print("\nEjemplo de datos:")
            print(df_reporte_pudo.head(10))
        else:
            print("No se generaron registros para el reporte de PUDOs.")


--- Paso 6.1: Generando reporte por PUDO (incluyendo Partners) ---
Cargando Partners desde CSV...
   Partners cargados: 649
Total puntos a analizar (PUDOs + Partners): 4073
   Procesando comuna 20/329: RENGO
   Procesando comuna 40/329: QUINTA NORMAL
   Procesando comuna 60/329: CHIGUAYANTE
   Procesando comuna 80/329: ANGOL
   Procesando comuna 100/329: COIHUECO
   Procesando comuna 120/329: LO ESPEJO
   Procesando comuna 140/329: COYHAIQUE
   Procesando comuna 160/329: CHIMBARONGO
   Procesando comuna 180/329: MARIA PINTO
   Procesando comuna 200/329: TALAGANTE
   Procesando comuna 220/329: PAREDONES
   Procesando comuna 240/329: SAN JUAN DE LA COSTA
   Procesando comuna 260/329: LUMACO
   Procesando comuna 280/329: PEUMO
   Procesando comuna 300/329: PUMANQUE
   Procesando comuna 320/329: SANTA BARBARA

✅ Archivo generado: Cobertura_Por_PUDO_y_Partners.xlsx
   Total registros: 4,073
   - PUDOs: 3,424
   - Partners: 649

Ejemplo de datos:
   PUDO_ID  AGENCY_ID                        

In [6]:
# ==========================================
# 7. ANÁLISIS DE OPORTUNIDADES (DONDE MOVER LA AGUJA)
# ==========================================

print("--- Paso 7: Análisis de Oportunidades de Expansión ---")

# Contexto: Buscamos maximizar el impacto. 
# "Mover la aguja" significa capturar la mayor cantidad de población nueva con el menor esfuerzo (nuevos puntos).
# Estrategia: Identificar Entidades (Barrios/Aldeas/Ciudades) con alta población NO cubierta.

if 'df_resumen' in locals() and not df_resumen.empty:
    
    # 1. Calcular el GAP (Población Sin Cubrir)
    df_oportunidades = df_resumen.copy()
    df_oportunidades['POBLACION_SIN_CUBRIR'] = df_oportunidades['POBLACION_TOTAL'] - df_oportunidades['POBLACION_CUBIERTA']
    
    # 2. Clasificar el tipo de oportunidad
    def clasificar_oportunidad(row):
        if row['CANTIDAD_PUDOS_LOCALIDAD'] == 0:
            return "EXPANSIÓN (Zona Nueva)"
        elif row['PORCENTAJE_COBERTURA'] < 0.5:
            return "DENSIFICACIÓN CRÍTICA (Cobertura < 50%)"
        elif row['PORCENTAJE_COBERTURA'] < 0.8:
            return "DENSIFICACIÓN MEDIA (Mejorar Servicio)"
        else:
            return "MANTENIMIENTO (Cobertura Alta)"

    df_oportunidades['TIPO_OPORTUNIDAD'] = df_oportunidades.apply(clasificar_oportunidad, axis=1)
    
    # 3. Ranking de Impacto
    # Ordenamos por la cantidad absoluta de personas que hoy NO atendemos
    df_ranking = df_oportunidades.sort_values(by='POBLACION_SIN_CUBRIR', ascending=False)
    
    # Filtramos casos triviales (ej. donde falta muy poca gente)
    df_ranking = df_ranking[df_ranking['POBLACION_SIN_CUBRIR'] > 100]

    # 4. Generar Reporte Estratégico
    cols_estrategia = [
        'COMUNA', 'LOCALIDAD',
        'POBLACION_TOTAL', 'POBLACION_CUBIERTA', 'POBLACION_SIN_CUBRIR', 
        'PORCENTAJE_COBERTURA', 'CANTIDAD_PUDOS_LOCALIDAD', 'TIPO_OPORTUNIDAD'
    ]
    
    df_estrategia = df_ranking[cols_estrategia]
    
    # Guardar
    filename_opp = 'Oportunidades_Expansion_PUDOs.xlsx'
    df_estrategia.to_excel(filename_opp, index=False)
    print(f"Reporte de Oportunidades generado: {filename_opp}")
    
    # --- VISUALIZACIÓN DE INSIGHTS ---
    print("\n=== TOP 10 LUGARES PARA 'MOVER LA AGUJA' (Mayor Población Sin Atender) ===")
    print(df_estrategia.head(10).to_string(index=False))
    
    print("\n=== TOP 5 OPORTUNIDADES 'GREENFIELD' (Lugares con 0 PUDOs y mucha gente) ===")
    greenfield = df_estrategia[df_estrategia['CANTIDAD_PUDOS_LOCALIDAD'] == 0]
    print(greenfield.head(5).to_string(index=False))

else:
    print("Error: No se encontró 'df_resumen'. Ejecuta el paso 6 primero.")

--- Paso 7: Análisis de Oportunidades de Expansión ---
Reporte de Oportunidades generado: Oportunidades_Expansion_PUDOs.xlsx

=== TOP 10 LUGARES PARA 'MOVER LA AGUJA' (Mayor Población Sin Atender) ===
       COMUNA                                LOCALIDAD  POBLACION_TOTAL  POBLACION_CUBIERTA  POBLACION_SIN_CUBRIR  PORCENTAJE_COBERTURA  CANTIDAD_PUDOS_LOCALIDAD                        TIPO_OPORTUNIDAD
  ANTOFAGASTA                              ANTOFAGASTA           387247              264765                122482                0.6837                        39  DENSIFICACIÓN MEDIA (Mejorar Servicio)
 VIÑA DEL MAR                          GRAN VALPARAÍSO           330109              225794                104315                0.6840                        60  DENSIFICACIÓN MEDIA (Mejorar Servicio)
   VALPARAÍSO                          GRAN VALPARAÍSO           221330              123049                 98281                0.5560                        34  DENSIFICACIÓN MEDIA (Mejorar S

In [7]:
# ==========================================
# 8. MAPA INTERACTIVO DE COBERTURA NACIONAL
# ==========================================

print("--- Paso 8: Generando Mapa Interactivo Nacional ---")

import folium
from folium.plugins import HeatMap, MarkerCluster, MiniMap, Search
from branca.colormap import LinearColormap
import numpy as np
import pandas as pd # Ensure pandas is available

# ==========================================
# A. PREPARACIÓN DE DATOS PARA EL MAPA
# ==========================================

print("Preparando datos geoespaciales...")

# 1. Convertir manzanas a EPSG:4326 (lat/lon) para Folium
manzanas_wgs84 = manzanas_utm.to_crs(epsg=4326)

# 2. Calcular centroides de cada manzana
manzanas_wgs84['centroid'] = manzanas_wgs84.geometry.centroid
manzanas_wgs84['lat'] = manzanas_wgs84['centroid'].y
manzanas_wgs84['lon'] = manzanas_wgs84['centroid'].x

# 3. Unir con métricas calculadas (df_final tiene pct_cobertura)
# Usar MANZENT como key
df_map = manzanas_wgs84[['MANZENT', 'COMUNA', 'ENTIDAD', 'LOCALIDAD', 'TOTAL_PERS', 'lat', 'lon']].copy()
df_map = df_map.merge(
    df_final[['MANZENT', 'pct_cobertura', 'pob_cubierta', 'pudos_en_manzana']],
    on='MANZENT',
    how='left'
 )
df_map['pct_cobertura'] = df_map['pct_cobertura'].fillna(0)
df_map['pob_cubierta'] = df_map['pob_cubierta'].fillna(0)
df_map['pudos_en_manzana'] = df_map['pudos_en_manzana'].fillna(0)

# 4. Calcular población sin cubrir
df_map['pob_sin_cubrir'] = df_map['TOTAL_PERS'] - df_map['pob_cubierta']
df_map['pob_sin_cubrir'] = df_map['pob_sin_cubrir'].clip(lower=0)

# 5. Filtrar manzanas con población > 0 para el heatmap
df_map_poblado = df_map[df_map['TOTAL_PERS'] > 0].copy()

print(f"Manzanas con población: {len(df_map_poblado):,}")

# ==========================================
# A.1. CARGA DE DATOS DE FORECAST
# ==========================================

print("Cargando datos de Forecast (Capacidad)...")

try:
    df_forecast = pd.read_excel('data/FCST intermedio paara heatmap.xlsx')
    
    # Normalizar nombre de comuna para hacer join
    df_forecast['COMUNA_NORM'] = df_forecast['comuna'].apply(normalize_text)

    # Crear dataframe de coordenadas por comuna (promedio de centroides)
    df_coords_comuna = df_map.groupby('COMUNA').agg({
        'lat': 'mean',
        'lon': 'mean'
    }).reset_index()
    df_coords_comuna['COMUNA_NORM'] = df_coords_comuna['COMUNA'].apply(normalize_text)
    
    # Join forecast con coordenadas
    df_forecast_geo = df_forecast.merge(
        df_coords_comuna[['COMUNA_NORM', 'lat', 'lon']],
        on='COMUNA_NORM',
        how='inner'
    )
    
    print(f"Comunas con datos de Forecast: {len(df_forecast_geo)}")
    
except Exception as e:
    print(f"Error cargando datos de Forecast: {e}")
    df_forecast_geo = pd.DataFrame()

# ==========================================
# B. CREAR MAPA BASE
# ==========================================

print("Creando mapa base...")

# Centro de Chile (aproximado)
chile_center = [-33.45, -70.65]

mapa = folium.Map(
    location=chile_center,
    zoom_start=6,
    tiles=None,
    control_scale=True
 )

folium.TileLayer('cartodbpositron', name='Mapa Claro').add_to(mapa)
folium.TileLayer('cartodbdark_matter', name='Mapa Oscuro').add_to(mapa)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(mapa)

# ==========================================
# C. CAPA 1: HEATMAP DE POBLACIÓN SIN COBERTURA
# ==========================================

print("Generando HeatMap de población sin cobertura...")

df_sin_cob = df_map_poblado[df_map_poblado['pob_sin_cubrir'] > 0].copy()
# Umbral más bajo pero aún filtrando rural disperso
df_sin_cob = df_sin_cob[df_sin_cob['pob_sin_cubrir'] >= 60]

df_sin_cob['log_pob'] = np.log10(df_sin_cob['pob_sin_cubrir'].clip(lower=1))

# Normalizar con percentil 99 para mantener ciudades altas visibles
q99 = df_sin_cob['log_pob'].quantile(0.99)
scale = q99 if q99 > 0 else 1.0
df_sin_cob['peso_norm'] = (df_sin_cob['log_pob'] / scale).clip(0, 1.0)

# Filtrar pesos bajos (menos transparencia)
df_sin_cob = df_sin_cob[df_sin_cob['peso_norm'] > 0.12]

heat_data_sin_cobertura = df_sin_cob[['lat', 'lon', 'peso_norm']].values.tolist()

print(f"   Puntos en heatmap: {len(heat_data_sin_cobertura):,}")
print(f"   Rango de pesos: {df_sin_cob['peso_norm'].min():.2f} - {df_sin_cob['peso_norm'].max():.2f}")

heatmap_sin_cobertura = HeatMap(
    heat_data_sin_cobertura,
    name='Población SIN Cobertura (Oportunidad)',
    min_opacity=0.05,
    max_zoom=18,
    radius=12,
    blur=8,
    gradient={
        0.0: 'transparent',
        0.12: 'transparent',
        0.2: 'yellow',
        0.4: 'orange',
        0.65: 'red',
        0.85: 'purple',
        1.0: 'darkred'
    }
 )
heatmap_sin_cobertura.add_to(mapa)

# ==========================================
# D. CAPA 2: HEATMAP DE POBLACIÓN CON COBERTURA (VERDE)
# ==========================================

print("Generando HeatMap de población cubierta...")

df_cubierta = df_map_poblado[df_map_poblado['pob_cubierta'] > 0].copy()
# Reestablecer escala original (solo verde) sin los cortes agresivos
df_cubierta = df_cubierta[df_cubierta['pob_cubierta'] >= 20]

df_cubierta['log_pob'] = np.log10(df_cubierta['pob_cubierta'].clip(lower=1))
q99_c = df_cubierta['log_pob'].quantile(0.99)
scale_c = q99_c if q99_c > 0 else 1.0
df_cubierta['peso_norm'] = (df_cubierta['log_pob'] / scale_c).clip(0, 1.0)
df_cubierta = df_cubierta[df_cubierta['peso_norm'] > 0.05]

heat_data_cubierta = df_cubierta[['lat', 'lon', 'peso_norm']].values.tolist()

heatmap_cubierta = HeatMap(
    heat_data_cubierta,
    name='Población CON Cobertura',
    min_opacity=0.2,
    max_zoom=18,
    radius=12,
    blur=8,
    gradient={
        0.0: 'palegreen',
        0.2: 'lightgreen',
        0.4: 'green',
        0.7: 'darkgreen',
        1.0: 'darkslategray'
    },
    show=False
 )
heatmap_cubierta.add_to(mapa)

# ==========================================
# E. CAPA 3: MARCADORES DE PUDOs (TIENDAS)
# ==========================================

print("Agregando marcadores de PUDOs...")

pudo_cluster = MarkerCluster(
    name='PUDOs Activos',
    show=True,
    options={
        'disableClusteringAtZoom': 14,
        'spiderfyOnMaxZoom': True
    }
 )

gdf_pudo_wgs84 = gdf_PUDO.to_crs(epsg=4326)

# Asegurar que las columnas numéricas sean float para evitar errores de formato
cols_numeric = [
    'q dao avg mes', 
    'q dao max mes', 
    'avg_packages_daily', 
    'peak_packages_weekly',
    'sobrecapacidad HOY - capacidad actual',
    'sobrecapacidad fcst peak - capacidad proyectada',
    'share del punto en la comuna'
]

for col in cols_numeric:
    if col in gdf_pudo_wgs84.columns:
        gdf_pudo_wgs84[col] = pd.to_numeric(gdf_pudo_wgs84[col], errors='coerce').fillna(0)

for idx, row in gdf_pudo_wgs84.iterrows():
    popup_html = f'''
    <div style='width:280px; font-family: Arial, sans-serif;'>
        <h4 style='margin:0 0 10px 0; color:#0066cc;'>{row.get('ofcn_dsc', 'PUDO')}</h4>
        <div style='font-size:11px; margin-bottom:5px;'><b>Comuna:</b> {row.get('cmns_nmb', 'N/A')}</div>
        <table style='width:100%; font-size:11px; border-collapse: collapse;'>
            <tr style='background-color:#f0f0f0;'>
                <td style='padding:4px;'><b>Tipo de Punto:</b></td>
                <td style='padding:4px;'>{row.get('tipo_punto', 'N/A')}</td>
            </tr>
            <tr>
                <td style='padding:4px;'><b>Apto Locker:</b></td>
                <td style='padding:4px;'>{row.get('apto_locker', 'N/A')}</td>
            </tr>
            <tr style='background-color:#f0f0f0;'>
                <td style='padding:4px;'><b>Tiene Locker:</b></td>
                <td style='padding:4px;'>{row.get('tiene_locker', 'N/A')}</td>
            </tr>
            <tr>
                <td style='padding:4px;'><b>Q DAO Avg Mes:</b></td>
                <td style='padding:4px;'>{row.get('q dao avg mes', 0):,.0f}</td>
            </tr>
            <tr style='background-color:#f0f0f0;'>
                <td style='padding:4px;'><b>Q DAO Max Mes:</b></td>
                <td style='padding:4px;'>{row.get('q dao max mes', 0):,.0f}</td>
            </tr>
            <tr>
                <td style='padding:4px;'><b>Avg Paquetes Diarios:</b></td>
                <td style='padding:4px;'>{row.get('avg_packages_daily', 0):,.1f}</td>
            </tr>
            <tr style='background-color:#f0f0f0;'>
                <td style='padding:4px;'><b>Peak Paquetes Semanal:</b></td>
                <td style='padding:4px;'>{row.get('peak_packages_weekly', 0):,.0f}</td>
            </tr>
             <tr>
                <td style='padding:4px;'><b>Sobrecapacidad Actual:</b></td>
                <td style='padding:4px;'>{row.get('sobrecapacidad HOY - capacidad actual', 0):,.0f}</td>
            </tr>
            <tr style='background-color:#f0f0f0;'>
                <td style='padding:4px;'><b>Sobrecapacidad Proyectada:</b></td>
                <td style='padding:4px;'>{row.get('sobrecapacidad fcst peak - capacidad proyectada', 0):,.0f}</td>
            </tr>
            <tr>
                <td style='padding:4px;'><b>Share en Comuna:</b></td>
                <td style='padding:4px;'>{row.get('share del punto en la comuna', 0)*100:.1f}%</td>
            </tr>
        </table>
    </div>
    '''

    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='blue', icon='store', prefix='fa')
    ).add_to(pudo_cluster)

pudo_cluster.add_to(mapa)

# ==========================================
# E.1. CAPA 3.1: PARTNERS DESDE CSV
# ==========================================

print("Agregando marcadores de Partners...")

try:
    df_partners = pd.read_csv('data/partners_mapa.csv')
    
    # Crear FeatureGroup para partners
    fg_partners = folium.FeatureGroup(name='Partners', show=False)
    
    for idx, row in df_partners.iterrows():
        partner_name = row.get('Partner/Tipo', 'Partner')
        lat = row.get('lat')
        lon = row.get('lon')
        
        if pd.notna(lat) and pd.notna(lon):
            folium.Marker(
                location=[lat, lon],
                popup=f"<b>{partner_name}</b>",
                icon=folium.Icon(color='green', icon='shopping-cart', prefix='fa')
            ).add_to(fg_partners)
    
    fg_partners.add_to(mapa)
    print(f"   {len(df_partners)} marcadores de partners agregados")
    
except Exception as e:
    print(f"Error cargando partners: {e}")

# ==========================================
# E.2. CAPA 3.5: FORECAST DE CAPACIDAD POR COMUNA
# ==========================================

print("Agregando marcadores de Forecast (Capacidad)...")

if not df_forecast_geo.empty:
    fg_forecast = folium.FeatureGroup(name='Forecast Capacidad Semanal', show=False)
    
    for idx, row in df_forecast_geo.iterrows():
        # Determinar color según Diferencia FCST Peak vs Capacidad Proyectada
        dif_fcst_peak = row.get('Diferencia FCST Peak vs Capacidad proyectada', 0)
        
        if dif_fcst_peak < -400:
            pin_color = 'darkpurple'
            icon_name = 'battery-empty'
        elif dif_fcst_peak < -200:
            pin_color = 'red'
            icon_name = 'exclamation-triangle'
        elif dif_fcst_peak < -100:
            pin_color = 'orange'
            icon_name = 'warning'
        else:  # Between -100 and positive
            pin_color = 'green'
            icon_name = 'check-circle'
        
        # Construir popup con toda la información
        popup_html = f"""
        <div style='width:280px; font-family: Arial, sans-serif;'>
            <h4 style='margin:0 0 10px 0; color:#0066cc;'>{row['comuna']}</h4>
            <table style='width:100%; font-size:11px; border-collapse: collapse;'>
                <tr style='background-color:#f0f0f0;'>
                    <td style='padding:4px;'><b>Ranking:</b></td>
                    <td style='padding:4px;'>{row.get('Ranking', 'N/A')}</td>
                </tr>
                <tr>
                    <td style='padding:4px;'><b>Capacidad Semanal:</b></td>
                    <td style='padding:4px;'>{row.get('Capacidad Semanal', 0):,.0f}</td>
                </tr>
                <tr style='background-color:#f0f0f0;'>
                    <td style='padding:4px;'><b>Capacidad Proyectada 10%:</b></td>
                    <td style='padding:4px;'>{row.get('Capacidad Proyectada 10%', 0):,.0f}</td>
                </tr>
                <tr>
                    <td style='padding:4px;'><b>OS Week FCST Valle:</b></td>
                    <td style='padding:4px;'>{row.get('OS Week FCST Valle', 0):,.1f}</td>
                </tr>
                <tr style='background-color:#f0f0f0;'>
                    <td style='padding:4px;'><b>OS Week FCST Peak:</b></td>
                    <td style='padding:4px;'>{row.get('OS Week FCST Peak', 0):,.0f}</td>
                </tr>
                <tr>
                    <td style='padding:4px;'><b>Total Agencias:</b></td>
                    <td style='padding:4px;'>{row.get('Total Agencias', 0)}</td>
                </tr>
                <tr style='background-color:#f0f0f0;'>
                    <td style='padding:4px;'><b>#Agencias Sobrecapacidad:</b></td>
                    <td style='padding:4px;'>{row.get('#Agencias Sobrecapacidad', 0)}</td>
                </tr>
                <tr>
                    <td style='padding:4px;'><b>% Sobrecapacidad:</b></td>
                    <td style='padding:4px; font-weight:bold;'>
                        {row.get('% Sobrecapacidad', 0)*100:.1f}%
                    </td>
                </tr>
                <tr style='background-color:#f0f0f0;'>
                    <td style='padding:4px;'><b>% Capacidad Peak:</b></td>
                    <td style='padding:4px;'>{row.get('%Capacidad Peak', 0)*100:.1f}%</td>
                </tr>
                <tr>
                    <td style='padding:4px;'><b>Dif. FCST Valle vs Cap:</b></td>
                    <td style='padding:4px;'>{row.get('Diferencia FCST Valle vs Capacidad actual', 0):,.0f}</td>
                </tr>
                <tr style='background-color:#f0f0f0;'>
                    <td style='padding:4px;'><b>Dif. FCST Peak vs Cap Proy:</b></td>
                    <td style='padding:4px; font-weight:bold; color:{"purple" if dif_fcst_peak < -400 else "red" if dif_fcst_peak < -200 else "orange" if dif_fcst_peak < -100 else "green"};'>
                        {dif_fcst_peak:,.0f}
                    </td>
                </tr>
                <tr>
                    <td style='padding:4px;'><b>Q puntos óptimo valle:</b></td>
                    <td style='padding:4px;'>{row.get('Q puntos optimizado valle', 0)}</td>
                </tr>
                <tr style='background-color:#f0f0f0;'>
                    <td style='padding:4px;'><b>Q puntos óptimo peak:</b></td>
                    <td style='padding:4px;'>{row.get('Q puntos optimizado peak', 0)}</td>
                </tr>
            </table>
        </div>
        """
        
        folium.Marker(
            location=[row['lat'], row['lon']],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color=pin_color, icon=icon_name, prefix='fa')
        ).add_to(fg_forecast)
    
    fg_forecast.add_to(mapa)
    print(f"   {len(df_forecast_geo)} marcadores de forecast agregados")
else:
    print("   No se pudieron agregar marcadores de forecast")

# ==========================================
# F. CAPA 4: TOP OPORTUNIDADES (Círculos grandes)
# ==========================================

print("Agregando TOP oportunidades de expansión...")

if 'df_estrategia' in locals() and not df_estrategia.empty:
    fg_oportunidades = folium.FeatureGroup(name='TOP Oportunidades Expansión', show=False)
    top_50 = df_estrategia.head(50)

    for idx, row in top_50.iterrows():
        # Buscar coordenadas por Comuna y Localidad (MATCH MÁS ESTRICTO)
        mask = (
            (df_map['LOCALIDAD'] == row['LOCALIDAD']) &
            (df_map['COMUNA'] == row['COMUNA'])
        )
        subset = df_map[mask]
        
        if not subset.empty:
            lat_mean = subset['lat'].mean()
            lon_mean = subset['lon'].mean()
            radius = min(max(row['POBLACION_SIN_CUBRIR'] / 100, 500), 5000)
            if row['CANTIDAD_PUDOS_LOCALIDAD'] == 0:
                color = 'red'
                tipo = 'NUEVA ZONA'
            elif row['PORCENTAJE_COBERTURA'] < 0.5:
                color = 'orange'
                tipo = 'CRITICO'
            else:
                color = 'yellow'
                tipo = 'MEJORAR'
            popup_html = f"""
            <div style='width:200px'>
                <h4>{row['LOCALIDAD']}</h4>
                <b>Comuna:</b> {row['COMUNA']}<br>
                <b>Población Total:</b> {row['POBLACION_TOTAL']:,}<br>
                <b>Sin Cubrir:</b> {row['POBLACION_SIN_CUBRIR']:,}<br>
                <b>Cobertura:</b> {row['PORCENTAJE_COBERTURA']*100:.1f}%<br>
                <b>PUDOs actuales:</b> {row['CANTIDAD_PUDOS_LOCALIDAD']}<br>
                <hr>
                <b style='color:{color}'>{tipo}</b>
            </div>
            """
            folium.CircleMarker(
                location=[lat_mean, lon_mean],
                radius=10,
                color=color,
                fill=True,
                fillColor=color,
                fillOpacity=0.7,
                popup=folium.Popup(popup_html, max_width=250)
            ).add_to(fg_oportunidades)

    fg_oportunidades.add_to(mapa)

# ==========================================
# G. CAPA 5: ZONAS SOBRESERVIDAS (Para optimizar)
# ==========================================

print("Identificando zonas sobreservidas...")

df_sobreservido = df_map_poblado[
    (df_map_poblado['pudos_en_manzana'] > 3) &
    (df_map_poblado['pct_cobertura'] > 0.9)
 ]

if len(df_sobreservido) > 0:
    fg_sobreservido = folium.FeatureGroup(name='Zonas Sobreservidas (Optimizar)', show=False)
    localidad_groups = df_sobreservido.groupby(['COMUNA', 'LOCALIDAD'])
    count_added = 0
    for (comuna, localidad), subset in localidad_groups:
        if len(subset) > 3 and count_added < 40:
            lat_mean = subset['lat'].mean()
            lon_mean = subset['lon'].mean()
            folium.CircleMarker(
                location=[lat_mean, lon_mean],
                radius=8,
                color='blue',
                fill=True,
                fillColor='blue',
                fillOpacity=0.5,
                popup=f"{localidad}<br>Comuna: {comuna}<br>Alta densidad de PUDOs"
            ).add_to(fg_sobreservido)
            count_added += 1
    fg_sobreservido.add_to(mapa)

# ==========================================
# H. AGREGAR CONTROLES Y LEYENDA
# ==========================================

print("Agregando controles...")

folium.LayerControl(collapsed=False).add_to(mapa)

# ==========================================
# H.1. AGREGAR BARRA DE BÚSQUEDA
# ==========================================

print("Agregando barra de búsqueda...")

import json

# Construir diccionario maestro de búsqueda
search_dict = {}

# 1. Comunas (desde Forecast si existe, o desde df_map)
if 'df_forecast_geo' in locals() and not df_forecast_geo.empty:
    for idx, row in df_forecast_geo.iterrows():
        search_dict[row['comuna']] = [row['lat'], row['lon']]
else:
    # Fallback: centroids de Comuna desde df_map
    c_centers = df_map.groupby('COMUNA')[['lat', 'lon']].mean().reset_index()
    for _, row in c_centers.iterrows():
        search_dict[row['COMUNA']] = [row['lat'], row['lon']]

# 2. Localidades (Implementación de búsqueda por Comuna + Localidad)
print("   Indexando Localidades (Comuna + Localidad) para el buscador...")
# Agrupar df_map por (Comuna, Localidad) para obtener centroide único
loc_centers = df_map.groupby(['COMUNA', 'LOCALIDAD'])[['lat', 'lon']].mean().reset_index()

for idx, row in loc_centers.iterrows():
    # Crear etiqueta combinada
    label = f"{row['LOCALIDAD']} ({row['COMUNA']})"
    search_dict[label] = [row['lat'], row['lon']]

print(f"   Total de entradas en el buscador: {len(search_dict)}")

if search_dict:
    # Javascript personalizado para la búsqueda
    # Usamos 'comunas' como nombre de variable en JS para mantener estructura, pero contiene Localidades también.
    
    # Get map ID
    map_id = mapa.get_name()
    
    search_html = f"""
    <div id="search-box" style="
        position: fixed;
        top: 10px;
        left: 60px;
        z-index: 1000;
        background-color: white;
        padding: 10px;
        border-radius: 5px;
        box-shadow: 0 2px 6px rgba(0,0,0,0.3);
        font-family: sans-serif;
    ">
        <input type="text" id="comuna-search" list="comunas-list" 
               placeholder="Buscar Comuna o Localidad..." 
               style="width: 250px; padding: 5px; border: 1px solid #ccc; border-radius: 3px;">
        <datalist id="comunas-list">
            {''.join([f'<option value="{k}">' for k in search_dict.keys()])}
        </datalist>
        <button onclick="searchAction()" style="
            padding: 5px 10px;
            background-color: #0066cc;
            color: white;
            border: none;
            border-radius: 3px;
            cursor: pointer;
            margin-left: 5px;
        ">Ir</button>
    </div>
    
    <script>
        var locations = {json.dumps(search_dict)};
        
        function searchAction() {{
            var input = document.getElementById('comuna-search').value.trim();
            
            if (!input) {{
                alert('Por favor ingrese un nombre.');
                return;
            }}
            
            var targetCoords = locations[input];
            
            // Si no hay match exacto, intentar búsqueda parcial (case-insensitive)
            if (!targetCoords) {{
                var lowerInput = input.toLowerCase();
                for (var key in locations) {{
                    if (key.toLowerCase() === lowerInput) {{
                        targetCoords = locations[key];
                        break;
                    }}
                }}
            }}
            
            if (targetCoords) {{
                var mapObj = window['{map_id}'];
                if (mapObj) {{
                    mapObj.setView(targetCoords, 14); // Zoom más cercano para localidades
                }}
            }} else {{
                alert('Lugar no encontrado en el índice.');
            }}
        }}
        
        document.getElementById('comuna-search').addEventListener('keypress', function(e) {{
            if (e.key === 'Enter') {{
                searchAction();
            }}
        }});
    </script>
    """
    
    mapa.get_root().html.add_child(folium.Element(search_html))
    print("   Barra de búsqueda agregada (Comunas y Localidades).")

# ==========================================
# I. GUARDAR MAPA
# ==========================================

filename_mapa = 'Mapa_Cobertura_Nacional_Blue.html'
mapa.save(filename_mapa)

print(f"\n✅ Mapa interactivo generado: {filename_mapa}")
print(f"   Tamaño aproximado del archivo: {len(heat_data_sin_cobertura):,} puntos de calor")
print("\nInstrucciones: abra el HTML, use panel de capas y haga zoom para detalle.")


--- Paso 8: Generando Mapa Interactivo Nacional ---
Preparando datos geoespaciales...
Manzanas con población: 191,061
Cargando datos de Forecast (Capacidad)...
Comunas con datos de Forecast: 312
Creando mapa base...
Generando HeatMap de población sin cobertura...
   Puntos en heatmap: 32,081
   Rango de pesos: 0.61 - 1.00
Generando HeatMap de población cubierta...
Agregando marcadores de PUDOs...
Agregando marcadores de Partners...
   659 marcadores de partners agregados
Agregando marcadores de Forecast (Capacidad)...
   312 marcadores de forecast agregados
Agregando TOP oportunidades de expansión...
Identificando zonas sobreservidas...
Agregando controles...
Agregando barra de búsqueda...
   Indexando Localidades (Comuna + Localidad) para el buscador...
   Total de entradas en el buscador: 9299
   Barra de búsqueda agregada (Comunas y Localidades).

✅ Mapa interactivo generado: Mapa_Cobertura_Nacional_Blue.html
   Tamaño aproximado del archivo: 32,081 puntos de calor

Instrucciones: a

In [8]:
gdf_PUDO

,esto_seq_cdg,ofcn_dsc,cmns_nmb,geol_latitud,geol_longitud,tipo_punto,apto_locker,tiene_locker,q dao avg mes,q dao max mes,avg_packages_daily,peak_packages_weekly,sobrecapacidad HOY - capacidad actual,sobrecapacidad fcst peak - capacidad proyectada,share del punto en la comuna,geometry,COMUNA_NORM
0,1,DROP OFF BX LIMITADA PRUEBA PROD,PUENTE ALTO,-33.564457,-70.545033,partner,si,no,37,54,4,38,1,1,2%,POINT (-70.54503 -33.56446),PUENTE ALTO
1,2,Centro Drop Off Paseo Bulnes,SANTIAGO,-33.446105,-70.653481,-,-,-,-,-,-,-,-,-,-,POINT (-70.65348 -33.4461),SANTIAGO
2,28,Punto Blue Express Cyber Don Francis,SANTIAGO,-33.446482,-70.645693,blue,no,no,30,51,2,35,1,0,1%,POINT (-70.64569 -33.44648),SANTIAGO
3,29,Centro Drop Off Portugal,SANTIAGO,-33.463074,-70.631230,blue,no,no,26,47,2,16,0,0,1%,POINT (-70.63123 -33.46307),SANTIAGO
4,33,Punto Blue Express Almacen Belorado,NUNOA,-33.450194,-70.620355,blue,no,no,18,27,2,12,0,0,1%,POINT (-70.62035 -33.45019),NUNOA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3419,9995,Punto Blue Express Copec Pitrufquen,PITRUFQUEN,-38.983739,-72.640687,copec,si,si,280,337,13,138,1,0,91%,POINT (-72.64069 -38.98374),PITRUFQUEN
3420,9996,Punto Blue Express Copec Lanco,LANCO,-39.449859,-72.780135,copec,si,si,71,88,4,28,0,0,18%,POINT (-72.78013 -39.44986),LANCO
3421,9997,Punto Blue Express Copec Nueva Imperial,NUEVA IMPERIAL,-38.748215,-72.951316,copec,si,si,471,503,21,166,0,1,74%,POINT (-72.95132 -38.74822),NUEVA IMPERIAL
3422,9998,Punto Blue Express Copec Gorbea,GORBEA,-39.098640,-72.672390,copec,si,si,254,298,11,101,0,1,70%,POINT (-72.67239 -39.09864),GORBEA
